## Vector Stores and Retrievers

In [9]:
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chat_models import init_chat_model

In [10]:
load_dotenv()

True

In [6]:
# Documents
documents = [
    Document(
        page_content="Dogs are greate companions, known for their loyality and friendliness",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"}
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around",
        metadata={"source": "mammal-pets-doc"}
    )
]

In [12]:
model = init_chat_model("gemma2-9b-it", model_provider="groq")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\LangGraph-Zero-to-Hero\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Vector Store
vector_store = Chroma.from_documents(documents, embeddings)

In [14]:
vector_store.similarity_search("cat")

[Document(id='d0ce75ad-f688-4470-89f9-7ef35879bc6c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(id='71c029a9-d88c-4e89-acf6-c8d6ab64ed49', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around'),
 Document(id='8fe61403-f1a8-4ea1-b2c9-d4875b920e53', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are greate companions, known for their loyality and friendliness'),
 Document(id='f88ed6f3-6423-417e-b801-97243bdfad7b', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [15]:
await vector_store.asimilarity_search("cat")

[Document(id='d0ce75ad-f688-4470-89f9-7ef35879bc6c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(id='71c029a9-d88c-4e89-acf6-c8d6ab64ed49', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around'),
 Document(id='8fe61403-f1a8-4ea1-b2c9-d4875b920e53', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are greate companions, known for their loyality and friendliness'),
 Document(id='f88ed6f3-6423-417e-b801-97243bdfad7b', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [16]:
vector_store.similarity_search_with_score("Dog")

[(Document(id='8fe61403-f1a8-4ea1-b2c9-d4875b920e53', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are greate companions, known for their loyality and friendliness'),
  1.1274324655532837),
 (Document(id='d0ce75ad-f688-4470-89f9-7ef35879bc6c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
  1.4828627109527588),
 (Document(id='71c029a9-d88c-4e89-acf6-c8d6ab64ed49', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around'),
  1.646347999572754),
 (Document(id='f88ed6f3-6423-417e-b801-97243bdfad7b', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  1.6571112871170044)]

### Retrievers

In [21]:
from langchain_core.runnables import RunnableLambda

In [24]:
# Method 1: Not that good
retriever = RunnableLambda(vector_store.similarity_search).bind(k=1)
retriever.batch(["cat", "dog"])

[[Document(id='d0ce75ad-f688-4470-89f9-7ef35879bc6c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(id='8fe61403-f1a8-4ea1-b2c9-d4875b920e53', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are greate companions, known for their loyality and friendliness')]]

In [26]:
# Method 2: Best method
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 1
    }
)

In [28]:
retriever.batch(["cat", "dog"])

[[Document(id='d0ce75ad-f688-4470-89f9-7ef35879bc6c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(id='8fe61403-f1a8-4ea1-b2c9-d4875b920e53', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are greate companions, known for their loyality and friendliness')]]

## Retriever with Chain

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [30]:
template = (
"""
Answer the question using the provided context only.

Question: {question}
Context: {context}
"""
)
prompt = ChatPromptTemplate.from_messages([("human", template)])

In [32]:
# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [36]:
chain.invoke("Tell me about dogs")

'Dogs are great companions, known for their loyalty and friendliness.  \n'